In [97]:
import os
import re
import urllib.request
from datetime import datetime
import pandas as pd
import numpy as np

In [98]:
def download_vhi_data(start_year=1981, end_year=2024):
    
    base_url = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php"
    os.makedirs("data", exist_ok=True)

    for province_id in range(1, 26):

        # перевірка існування файлу
        existing = [f for f in os.listdir("data") if f.startswith(f"VHI_{province_id}_")]
        if existing:
            print(f"Область {province_id} вже завантажена.")
            continue

        url = (
            f"{base_url}?country=UKR&provinceID={province_id}"
            f"&year1={start_year}&year2={end_year}&type=Mean"
        )

        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        filename = f"data/VHI_{province_id}_{timestamp}.csv"

        try:
            urllib.request.urlretrieve(url, filename)
            print(f"Завантажено область {province_id}")
        except Exception as e:
            print(f"Помилка для області {province_id}: {e}")

In [99]:
def clean_vhi_file(filepath, province_id):

    with open(filepath, 'r') as f:
        lines = f.readlines()

    # залишаємо тільки рядки що починаються з числа (рік)
    data_lines = [line for line in lines if re.match(r'^\d', line)]

    # розбиваємо і беремо тільки перші 7 колонок
    rows = [line.strip().split(',')[:7] for line in data_lines]

    df = pd.DataFrame(rows, columns=["Year", "Week", "SMN", "SMT", "VCI", "TCI", "VHI"])

    # переводимо в числа
    df = df.apply(pd.to_numeric, errors='coerce')

    # прибираємо NaN
    df = df.dropna()

    df["NOAA_ID"] = province_id

    return df

In [100]:
def load_all_data():
    all_data = []

    for file in os.listdir("data"):
        if file.endswith(".csv"):
            province_id = int(file.split("_")[1])
            filepath = os.path.join("data", file)

            df = clean_vhi_file(filepath, province_id)
            all_data.append(df)

    return pd.concat(all_data, ignore_index=True)

In [101]:
# порядок областей за українською абеткою
ukrainian_regions = [
    "Вінницька","Волинська","Дніпропетровська","Донецька","Житомирська",
    "Закарпатська","Запорізька","Івано-Франківська","Київська","Кіровоградська",
    "Крим", "Луганська","Львівська","Миколаївська","Одеська","Полтавська",
    "Рівненська","Сумська","Тернопільська","Харківська","Херсонська",
    "Хмельницька","Черкаська","Чернівецька","Чернігівська"
]

# створюємо нову українську нумерацію
ua_index = {name: i+1 for i, name in enumerate(ukrainian_regions)}

# відповідність NOAA → назва області
noaa_to_name = {
    1: "Черкаська",
    2: "Чернігівська",
    3: "Чернівецька",
    4: "Крим",
    5: "Дніпропетровська",
    6: "Донецька",
    7: "Івано-Франківська",
    8: "Харківська",
    9: "Херсонська",
    10: "Хмельницька",
    11: "Київська",
    12: "Кіровоградська",
    13: "Луганська",
    14: "Львівська",
    15: "Миколаївська",
    16: "Одеська",
    17: "Полтавська",
    18: "Рівненська",
    19: "Сумська",
    20: "Тернопільська",
    21: "Закарпатська",
    22: "Вінницька",
    23: "Волинська",
    24: "Житомирська",
    25: "Запорізька"
}

In [102]:
def apply_ukrainian_indexing(df):

    df["Province_Name"] = df["NOAA_ID"].map(noaa_to_name)

    # створюємо новий український індекс
    df["UA_ID"] = df["Province_Name"].map(ua_index)

    return df

In [103]:
def get_vhi_by_year(df, region_id, year):
    return df[(df["UA_ID"] == region_id) & (df["Year"] == year)]

def get_vhi_range(df, regions, start_year, end_year):
    return df[
        (df["UA_ID"].isin(regions)) &
        (df["Year"] >= start_year) &
        (df["Year"] <= end_year)
    ]
    
def get_vhi_range(df, regions, start_year, end_year):
    return df[
        (df["UA_ID"].isin(regions)) &
        (df["Year"] >= start_year) &
        (df["Year"] <= end_year)
    ]

def get_drought_years(df, threshold=20):
    return df[df["VHI"] < threshold]

In [104]:
download_vhi_data()

df = load_all_data()

df = apply_ukrainian_indexing(df)

print("Дані успішно оброблені.")
df.head()

Область 1 вже завантажена.
Область 2 вже завантажена.
Область 3 вже завантажена.
Область 4 вже завантажена.
Область 5 вже завантажена.
Область 6 вже завантажена.
Область 7 вже завантажена.
Область 8 вже завантажена.
Область 9 вже завантажена.
Область 10 вже завантажена.
Область 11 вже завантажена.
Область 12 вже завантажена.
Область 13 вже завантажена.
Область 14 вже завантажена.
Область 15 вже завантажена.
Область 16 вже завантажена.
Область 17 вже завантажена.
Область 18 вже завантажена.
Область 19 вже завантажена.
Область 20 вже завантажена.
Область 21 вже завантажена.
Область 22 вже завантажена.
Область 23 вже завантажена.
Область 24 вже завантажена.
Область 25 вже завантажена.
Дані успішно оброблені.


,Year,Week,SMN,SMT,VCI,TCI,VHI,NOAA_ID,Province_Name,UA_ID
0,1982,2,0.063,261.53,55.89,38.20,47.04,10,Хмельницька,22
1,1982,3,0.063,263.45,57.30,32.69,44.99,10,Хмельницька,22
2,1982,4,0.061,265.10,53.96,28.62,41.29,10,Хмельницька,22
3,1982,5,0.058,266.42,46.87,28.57,37.72,10,Хмельницька,22
4,1982,6,0.056,267.47,39.55,30.27,34.91,10,Хмельницька,22


In [106]:
df[["Province_Name","UA_ID"]].drop_duplicates().sort_values("UA_ID")

,Province_Name,UA_ID
29055,Вінницька,1
31290,Волинська,2
44700,Дніпропетровська,3
46935,Донецька,4
33525,Житомирська,5
26820,Закарпатська,6
35760,Запорізька,7
49170,Івано-Франківська,8
2235,Київська,9
4470,Кіровоградська,10
